In [211]:
!pip3 install pillow
from PIL import Image
import numpy as np
import csv
import json

6907.64s - pydevd: Sending message related to process being replaced timed-out after 5 seconds

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip3 install --upgrade pip


In [169]:
def find_grid(num_images=100, folder='SymmStim'):
    result = []
    for i in range(1, num_images+1):
        # Load the image
        img = Image.open(f'{folder}/Slide{i}.BMP')
        
        # Convert the image to grayscale to simplify the process
        img = img.convert('L')

        # Convert the image data to a numpy array
        img_data = np.array(img)

        height, width = img_data.shape

        # Initialize start and end points
        start_point = None
        end_point = None

        # Iterate through each row
        for y in range(height):
            # Iterate through each column
            for x in range(width):
                # If we find a non-white pixel and haven't found a start point yet
                if img_data[y, x] <= 0 and start_point is None:
                    start_point = (x, y)
                # If we find a white pixel and we have found a start point
                elif img_data[y, x] > 0 and start_point is not None:
                    end_point = (x-1, y-1)
                    break
            # If we have found both start and end points, we can stop looking
            if start_point is not None and end_point is not None:
                break

        # Add the start and end points to the results
        result.append((start_point, end_point))
    return result


In [173]:
def make_multiple_areas_red(image_path, output_path, start_x, end_x, start_y, end_y):
    # Open the image file
    img = Image.open(image_path)

    # Make sure the image is in RGB mode
    if img.mode != 'RGB':
        img = img.convert('RGB')

    # Access the pixel data
    data = img.load()

    width, height = img.size

    # Iterate through the specified x and y ranges, 35 pixels apart
    for x in range(start_x, min(end_x, width-3), 33):
        for y in range(start_y, min(end_y, height-3), 33):
            # Check that the area fits within the image
            if x+1 > width or y+1 > height:
                raise ValueError("The specified area extends beyond the image boundaries")
            # Set the color of a 5x5 pixel area starting at (x, y) to red
            for i in range(x, min(x + 1, width)):
                for j in range(y, min(y + 1, height)):
                    data[i, j] = (255, 0, 0)  # RGB for red

    # Save the modified image
    img.save(output_path)


In [134]:
start_location = (231, 141)
end_location = (489, 400)
# Example usage:
make_multiple_areas_red('./SymmStim/Slide1.BMP', 'output.bmp', 240, 478, 145, 400)

In [149]:
def get_and_color_pixel(image_path, output_path, start_x, end_x, start_y, end_y):
    # Open the image file
    img = Image.open(image_path)

    # Make sure the image is in RGB mode
    if img.mode != 'RGB':
        img = img.convert('RGB')

    # Access the pixel data
    data = img.load()

    width, height = img.size

    original_pixel_values = []

    # Iterate through the specified x and y ranges, 33 pixels apart
    for x in range(start_x, min(end_x, width), 33):
        for y in range(start_y, min(end_y, height), 33):
            # Check that the pixel location fits within the image
            if x >= width or y >= height:
                raise ValueError("The specified pixel extends beyond the image boundaries")
            
            # Append the RGB value of the pixel at (x, y) to the list
            original_pixel_values.append(data[x, y])  
            
            # Color the pixel at (x, y) red
            data[x, y] = (255, 0, 0)

    # Save the modified image
    img.save(output_path)

    return original_pixel_values



In [199]:

def convert_traversal(array):
    # Reshape the 1D array into a 2D array with 8 columns
    grid = np.reshape(array, (-1, 8))

    # Transpose the 2D array so that rows become columns and vice versa
    grid_t = np.transpose(grid)

    # Flatten the transposed 2D array back into a 1D array
    new_array = grid_t.flatten()

    return new_array

In [212]:
all_colors = []
for i in range(1, 101):
    input_path = './SymmStim/Slide{}.BMP'.format(i)
    output_path = './data/Slide{}.BMP'.format(i)
    colors = get_and_color_pixel(input_path, output_path, 240, 478, 145, 400)
    colors_string = []
    for color in colors:
        if(color == (0,0,0)):
            colors_string.append('black')
        elif(color == (255,255,255)):
            colors_string.append('white')
    traversed_horizontal = convert_traversal(colors_string)
    all_colors.append(traversed_horizontal)

print(all_colors)

[array(['black', 'black', 'white', 'white', 'white', 'white', 'black',
       'black', 'white', 'white', 'black', 'black', 'black', 'black',
       'white', 'white', 'white', 'white', 'white', 'white', 'white',
       'white', 'white', 'white', 'black', 'black', 'white', 'white',
       'white', 'white', 'black', 'black', 'white', 'white', 'black',
       'black', 'black', 'black', 'white', 'white', 'white', 'black',
       'black', 'black', 'black', 'black', 'black', 'white', 'black',
       'black', 'black', 'white', 'white', 'black', 'black', 'black',
       'black', 'white', 'black', 'white', 'white', 'black', 'white',
       'black'], dtype='<U5'), array(['white', 'black', 'black', 'white', 'white', 'black', 'black',
       'white', 'black', 'black', 'black', 'white', 'white', 'black',
       'black', 'black', 'white', 'black', 'black', 'white', 'white',
       'black', 'black', 'white', 'white', 'black', 'white', 'white',
       'white', 'white', 'black', 'white', 'white', 'white

In [219]:
# Convert each numpy.ndarray to a Python list
python_list_as_lists = [arr.tolist() for arr in all_colors]

# Serialize the entire Python list to a JSON string
python_list_json = json.dumps(python_list_as_lists)
print(python_list_json)

# Specify the file path
file_path = "data.json"

# Write the JSON data to the file
with open(file_path, "w") as json_file:
    json_file.write(python_list_json)

[["black", "black", "white", "white", "white", "white", "black", "black", "white", "white", "black", "black", "black", "black", "white", "white", "white", "white", "white", "white", "white", "white", "white", "white", "black", "black", "white", "white", "white", "white", "black", "black", "white", "white", "black", "black", "black", "black", "white", "white", "white", "black", "black", "black", "black", "black", "black", "white", "black", "black", "black", "white", "white", "black", "black", "black", "black", "white", "black", "white", "white", "black", "white", "black"], ["white", "black", "black", "white", "white", "black", "black", "white", "black", "black", "black", "white", "white", "black", "black", "black", "white", "black", "black", "white", "white", "black", "black", "white", "white", "black", "white", "white", "white", "white", "black", "white", "white", "white", "black", "white", "white", "black", "white", "white", "white", "white", "black", "black", "black", "black", "white